# GDS to Meep

In GDSFactory, there are several functions that can be used to take a GDSFactory component and turn it into a Meep geometry or simulation. When used in conjunction with other GDSFactory functions that allow you to take any gds file and make its contents a GDSFactory component, this means that you can take any component and easily turn it into a Meep geometry.

* Description of what the functions do (more about how gds works and maybe give a little bit of detail about how the functions are coded)
* what keywords they accept
* pitfalls to avoid



In [ ]:
# Code showing examples for all the functions

# Library where the functions are from
from gdsfactory.simulation.gmeep.get_simulation import get_simulation, get_meep_geometry_from_component
# get_simulation

# get_meep_geometry_from_component

from gdsfactory.simulation.gmeep import write_sparameters_meep

### Finding S-parameters through gmeep

There are several functions in GDSFactory that allow you to find the S-parameters of a given component. 

### Adjoint solver in GDSFactory

The documentation on this in gdsfactory is slightly better and they have some example code that looks like it functions

However, it is still rather confined in GDSFactory, the functions there take a bit of finessing in order to have the same flexibility

* How the adjoint solver works with the gmeep plugin